In [ ]:
# Cell 1: Install Additional Packages
# Demonstrates pip install capability in Container Runtime notebooks
# Installing tabulate for better formatted output tables
# Reference: https://docs.snowflake.com/en/developer-guide/snowflake-ml/container-runtime-package-management

!pip install tabulate

print("Installed tabulate for formatted table output")

In [ ]:
# Cell 2: Import Libraries
import sys
import time
import uuid
from datetime import datetime
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session

print("=" * 80)
print("Carbon Emissions Calculator")
print("=" * 80)

In [ ]:
# Cell 3: Get Company Name from Parameters
# Development mode: Set COMPANY_NAME directly for interactive testing
# Production mode: Pass via ARGUMENTS parameter in EXECUTE NOTEBOOK PROJECT

# Default for development/testing
COMPANY_NAME = 'TechCorp'  # Change this to test different companies

# Override with command-line argument if provided (EXECUTE NOTEBOOK PROJECT)
if len(sys.argv) > 1:
    COMPANY_NAME = sys.argv[1]
    print(f"Using parameter: {COMPANY_NAME}")
else:
    print(f"Using default: {COMPANY_NAME} (interactive mode)")

execution_id = str(uuid.uuid4())
start_time = time.time()

print(f"Processing: {COMPANY_NAME}")
print(f"Execution ID: {execution_id}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()


In [ ]:
# Cell 4: Initialize Session
session = snowpark.context.get_active_session()

session.sql("USE DATABASE DEMO_DB").collect()
session.sql("USE SCHEMA PUBLIC").collect()

print("Connected to Snowflake")
print(f"  Database: {session.get_current_database()}")
print(f"  Schema: {session.get_current_schema()}")
print()

In [ ]:
# Cell 5: Load Company Activity Data
print(f"Loading activity data for {COMPANY_NAME}...")

activity_df = session.table("COMPANY_ACTIVITY_DATA") \
    .filter(f"COMPANY_NAME = '{COMPANY_NAME}'") \
    .to_pandas()

if len(activity_df) == 0:
    print(f"ERROR: No data found for '{COMPANY_NAME}'")
    print("\nAvailable companies:")
    available = session.sql("SELECT DISTINCT COMPANY_NAME FROM COMPANY_ACTIVITY_DATA ORDER BY COMPANY_NAME").to_pandas()
    print(available.to_string(index=False))
    sys.exit(1)

company_id = activity_df['COMPANY_ID'].iloc[0]
print(f"Found {len(activity_df)} activity records")
print()

In [ ]:
# Cell 6: Load Emissions Factors
print("Loading emissions factors...")
factors_df = session.table("EMISSIONS_FACTORS").to_pandas()
print(f"Loaded {len(factors_df)} conversion factors")
print()

In [ ]:
# Cell 7: Calculate Emissions
print("Calculating emissions...")

# Join activity data with conversion factors
merged_df = activity_df.merge(
    factors_df,
    on=['ACTIVITY_TYPE', 'SCOPE'],
    how='left'
)

# Check for missing factors
missing_factors = merged_df[merged_df['EMISSION_FACTOR'].isna()]
if len(missing_factors) > 0:
    print(f"WARNING: {len(missing_factors)} activities missing conversion factors")

# Calculate emissions
merged_df['EMISSIONS_KG_CO2E'] = merged_df['ACTIVITY_VALUE'] * merged_df['EMISSION_FACTOR']
merged_df['EMISSIONS_TCO2E'] = merged_df['EMISSIONS_KG_CO2E'] / 1000

print(f"Calculated emissions for {len(merged_df)} activities")
print()

In [ ]:
# Cell 8: Summary by Scope
print("Emissions Summary:")
print("-" * 60)

scope_summary = merged_df.groupby('SCOPE').agg({
    'EMISSIONS_TCO2E': 'sum'
}).round(2)

for scope, row in scope_summary.iterrows():
    print(f"   {scope:12} {row['EMISSIONS_TCO2E']:>12,.2f} tonnes CO2")

total_emissions = scope_summary['EMISSIONS_TCO2E'].sum()
print("-" * 60)
print(f"   {'TOTAL':12} {total_emissions:>12,.2f} tonnes CO2")
print()

In [ ]:
# Cell 9: Detailed Breakdown
from tabulate import tabulate

print("Detailed Emissions Breakdown:")
print()

detail_summary = merged_df.groupby(['SCOPE', 'ACTIVITY_TYPE']).agg({
    'ACTIVITY_VALUE': 'sum',
    'UNIT': 'first',
    'EMISSIONS_TCO2E': 'sum'
}).round(2).reset_index()

detail_summary = detail_summary.sort_values(['SCOPE', 'EMISSIONS_TCO2E'], ascending=[True, False])

# Format emissions with commas
detail_summary['EMISSIONS_DISPLAY'] = detail_summary['EMISSIONS_TCO2E'].apply(lambda x: f"{x:,.2f}")

# Use tabulate for nice formatting
print(tabulate(
    detail_summary[['SCOPE', 'ACTIVITY_TYPE', 'EMISSIONS_DISPLAY']],
    headers=['Scope', 'Activity Type', 'Emissions (tonnes CO2)'],
    tablefmt='grid',
    showindex=False
))
print()

In [ ]:
# Cell 10: Save Results
execution_time = time.time() - start_time

print("Saving results...")

results_data = []
for scope, row in scope_summary.iterrows():
    results_data.append({
        'EXECUTION_ID': execution_id,
        'COMPANY_ID': company_id,
        'COMPANY_NAME': COMPANY_NAME,
        'SCOPE': scope,
        'TOTAL_EMISSIONS_TCO2E': float(row['EMISSIONS_TCO2E']),
        'REPORTING_YEAR': 2024,
        'EXECUTION_TIME_SECONDS': execution_time,
        'PARAMETERS': f"company_name={COMPANY_NAME}"
    })

results_df = session.create_dataframe(results_data)
results_df.write.mode("append").save_as_table("EMISSIONS_RESULTS")

print("Results saved to EMISSIONS_RESULTS table")
print()

In [ ]:
# Cell 11: Done
print("=" * 80)
print("COMPLETE")
print("=" * 80)
print(f"Company: {COMPANY_NAME}")
print(f"Total Emissions: {total_emissions:,.2f} tonnes CO2")
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"Execution ID: {execution_id}")
print("=" * 80)